In [4]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [5]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "medical_history").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [6]:
from medminer.task.history import history_task

for doc in docs:
    history_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: history                                                                                              │
│ Prompt:                                                                                                         │
│                                                                                                                 │
│     Given a medical history of a patient, extract all given diagnoses and save all information as csv. The      │
│ diagnosis should be translated to english. The medical history is usually in the format of a sentence or a      │
│ paragraph. Every diagnosis should have a single row, if there are multiple diagnosis that can be extracted from │
│ a single piece of text, split them up. These are the steps you should follow to complete the task:              │
│     1. extract a part of the text that contains a diagnosis. The diagnosis can be in any language. This is the  │
│ diagnosis_reference column.                                                                                     │
│     2. translate the diagnoses to english if necessary and infer the diagnosis_translated column.               │
│     3. Extract the relevant diagnosis as a string and loose everything that is not relevant. This is the        │
│ diagnosis column.                                                                                               │
│     4. Extract the month and year of the medical history for that diagnosis. If not applicable, write an empty  │
│ string.                                                                                                         │
│                                                                                                                 │
│     Example 1:                                                                                                  │
│     Input: "schwere Herzinsuffizienz, 2. Grad, NYHA II, ED 07/2023"                                             │
│     Output: [{"patient_id": 1, "diagnosis_reference": "schwere chronische Herzinsuffizienz, 2. Grad, NYHA II",  │
│ "diagnosis_translated": "severe chronic heart failure, 2nd degree, NYHA II", "diagnosis": "chronic heart        │
│ failure", "month": "7", "year": "2023"}\]                                                                       │
│                                                                                                                 │
│     Example 2:                                                                                                  │
│     Input: "Myocardial Infarction"                                                                              │
│     Output: [{"patient_id": 2, "diagnosis_reference": "Myocardial Infarction", "diagnosis_translated":          │
│ "Myocardial Infarction", "diagnosis": "Myocardial Infarction", "month": "", "year": ""}\]                       │
│                                                                                                                 │
│     Example 3:                                                                                                  │
│     Input: "Lungenteilresektion rechts bei Lungenmetastasen eines Kolonkarzinoms, 2023-07-01"                   │
│     Output: [                                                                                                   │
│         {"patient_id": 3, "diagnosis_reference": "Lungenteilresektion rechts bei Lungenmetastasen eines         │
│ Kolonkarzinoms", "diagnosis_translated": "Lung resection due to lung metastases of colon cancer", "diagnosis":  │
│ "colon cancer", "month": "7", "year": "2023"},                                                                  │
│         {"patient_id": 3, "diagnosis_reference": "Lung

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'extract_diagnosis_data' with arguments: {'data': [{'patient_id': 3, 'diagnosis_reference':       │
│ 'koronare Zweigefäßerkrankung', 'diagnosis_translated': 'coronary branch artery disease', 'diagnosis':          │
│ 'coronary artery disease', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 'NSTEMI 04/2017', │
│ 'diagnosis_translated': 'NSTEMI', 'diagnosis': 'NSTEMI', 'month': '4', 'year': '2017'}, {'patient_id': 3,       │
│ 'diagnosis_reference': '90%ige Stenose des proximalen R. intraventicularis und 90-95%iger Stenose des           │
│ proximalen RCX', 'diagnosis_translated': '90% stenosis of the proximal intraventricular branch and 90-95%       │
│ stenosis of the proximal RCX', 'diagnosis': 'stenosis of coronary arteries', 'month': '', 'year': ''},          │
│ {'patient_id': 3, 'diagnosis_reference': 'hochgradig reduzierte LV-Funktion bei Akinesie der anterolateralen    │
│ Wand', 'diagnosis_translated': 'highly reduced left ventricular function with akinesia of the anterolateral     │
│ wall', 'diagnosis': 'left ventricular dysfunction', 'month': '', 'year': ''}, {'patient_id': 3,                 │
│ 'diagnosis_reference': 'Hypertonus', 'diagnosis_translated': 'hypertension', 'diagnosis': 'hypertension',       │
│ 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 'hypertensive Herzerkrankung',               │
│ 'diagnosis_translated': 'hypertensive heart disease', 'diagnosis': 'hypertensive heart disease', 'month': '',   │
│ 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 'chron. Niereninsuffizienz', 'diagnosis_translated':      │
│ 'chronic renal failure', 'diagnosis': 'chronic kidney disease', 'month': '', 'year': ''}]}                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'patient_id': 3, 'diagnosis_reference': 'koronare Zweigefäßerkrankung', 'diagnosis_translated': 
'coronary branch artery disease', 'diagnosis': 'coronary artery disease', 'month': '', 'year': ''}, {'patient_id': 
3, 'diagnosis_reference': 'NSTEMI 04/2017', 'diagnosis_translated': 'NSTEMI', 'diagnosis': 'NSTEMI', 'month': '4', 
'year': '2017'}, {'patient_id': 3, 'diagnosis_reference': '90%ige Stenose des proximalen R. intraventicularis und 
90-95%iger Stenose des proximalen RCX', 'diagnosis_translated': '90% stenosis of the proximal intraventricular 
branch and 90-95% stenosis of the proximal RCX', 'diagnosis': 'stenosis of coronary arteries', 'month': '', 'year':
''}, {'patient_id': 3, 'diagnosis_reference': 'hochgradig reduzierte LV-Funktion bei Akinesie der anterolateralen 
Wand', 'diagnosis_translated': 'highly reduced left ventricular function with akinesia of the anterolateral wall', 
'diagnosis': 'left ventricular dysfunction', 'month': '', 'year': ''}, {'patient_id': 3, 'diagnosis_reference': 
'Hypertonus', 'diagnosis_translated': 'hypertension', 'diagnosis': 'hypertension', 'month': '', 'year': ''}, 
{'patient_id': 3, 'diagnosis_reference': 'hypertensive Herzerkrankung', 'diagnosis_translated': 'hypertensive heart
disease', 'diagnosis': 'hypertensive heart disease', 'month': '', 'year': ''}, {'patient_id': 3, 
'diagnosis_reference': 'chron. Niereninsuffizienz', 'diagnosis_translated': 'chronic renal failure', 'diagnosis': 
'chronic kidney disease', 'month': '', 'year': ''}]

[Step 1: Duration 4.11 seconds| Input tokens: 2,171 | Output tokens: 376]

...

[Step 3: Duration 49.89 seconds| Input tokens: 16,078 | Output tokens: 1,186]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The medical history diagnoses have been successfully   │
│ extracted, translated, mapped to ICD-11 codes, and saved to history.csv.'}                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The medical history diagnoses have been successfully extracted, translated, mapped to ICD-11 codes, 
and saved to history.csv.

[Step 4: Duration 0.73 seconds| Input tokens: 27,192 | Output tokens: 1,224]